In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm

In [4]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [5]:
print("num of chars", len(text))

num of chars 1115394


In [6]:
# First 1000 chars
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [7]:
# All unique chars that occur
# We'all use char as token
chars = sorted(list(set(text)))
print(''.join(chars))
print(len(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [8]:
# Simple tokenizer example (just using index) (More complex, see OpenAI tiktoken)
# Write the encode and decode function
stoi = {character:index for index, character in enumerate(chars)}
itos = {index:character for index, character in enumerate(chars)}
encode = lambda chars: [stoi[x] for x in chars]
decode = lambda ints: [itos[x] for x in ints]

test_str = "From either sides the river lies"
test_str_code = encode(test_str)
test_str_decode = decode(test_str_code)
print(test_str_code)
print(test_str_decode)

[18, 56, 53, 51, 1, 43, 47, 58, 46, 43, 56, 1, 57, 47, 42, 43, 57, 1, 58, 46, 43, 1, 56, 47, 60, 43, 56, 1, 50, 47, 43, 57]
['F', 'r', 'o', 'm', ' ', 'e', 'i', 't', 'h', 'e', 'r', ' ', 's', 'i', 'd', 'e', 's', ' ', 't', 'h', 'e', ' ', 'r', 'i', 'v', 'e', 'r', ' ', 'l', 'i', 'e', 's']


In [9]:
# Encode the entire dataset
# Split the train and validation set of the dataset
# Transfer to tensor
data = torch.tensor(encode(text), dtype=torch.long)
split_point = int(0.9 * len(data))
train_data = data[:split_point]
test_data = data[split_point:]

In [10]:
# Write the get_batch function
# Draw chunks of data and understand how to use a chunk of data consider varying length

batch_size = 8
block_size = 8
device = "cuda" if torch.cuda.is_available() else "cpu"

def get_batch(data):
    indices = torch.randint(len(data)-block_size, (batch_size, ))
    x = [data[start:start+block_size] for start in indices]
    y = [data[start+1:start+block_size+1] for start in indices]
    x, y = torch.stack(x), torch.stack(y)
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch(train_data)
print(x)

tensor([[58, 43, 50, 63,  1, 51, 53, 56],
        [ 0, 40, 63,  1, 51, 47, 52, 43],
        [ 0, 20, 53, 61,  1, 41, 53, 59],
        [43, 56, 57, 58, 39, 52, 42, 47],
        [47, 52,  1, 53, 59, 56,  1, 44],
        [42,  1, 54, 56, 43, 57, 39, 45],
        [53, 59, 56,  8,  0, 37, 53, 59],
        [ 1, 39, 57,  1, 39, 52, 63,  1]])


In [11]:
# The way how a chunk of data is used: Enumerate all possible prediction context length
x_example = train_data[:block_size]
y_example = train_data[1:block_size+1]

for t in range(block_size):
    x = x_example[:t+1]
    y = y_example[t]
    print("----------------------")
    print("Training input is ", x)
    print("Target is ", y)

----------------------
Training input is  tensor([18])
Target is  tensor(47)
----------------------
Training input is  tensor([18, 47])
Target is  tensor(56)
----------------------
Training input is  tensor([18, 47, 56])
Target is  tensor(57)
----------------------
Training input is  tensor([18, 47, 56, 57])
Target is  tensor(58)
----------------------
Training input is  tensor([18, 47, 56, 57, 58])
Target is  tensor(1)
----------------------
Training input is  tensor([18, 47, 56, 57, 58,  1])
Target is  tensor(15)
----------------------
Training input is  tensor([18, 47, 56, 57, 58,  1, 15])
Target is  tensor(47)
----------------------
Training input is  tensor([18, 47, 56, 57, 58,  1, 15, 47])
Target is  tensor(58)


In [12]:
# Test the data using simple language model BigramLanguageModel
# Embedding in Bigram is just a nxn matrix represents the transition probability from token to token
"""
BigramLanguageModel
-------------------
Forward:
Input: x, tensor of training data with shape (B, T).
Input: target, tensor of labeling data with shape (B, T). Default target = None
Output: logits, rows of the probability of each token in the data x, shape (B, T, C), C is the length of the vocab.
Output: loss, cross entropy of logits and targets. Note that input logits have to be reshaped to use crossEntropyLoss.

Generate: write a generic version that considers the history
Input: x, tensor (B, T)
Input: max_new_tokens
Output: x' after expanding
"""

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, x, target=None):
        logits = self.token_embedding_table(x)
        
        if target is None:
            loss = 0
        else:
            B, T, C = logits.shape # C represents channels
            logits_reshaped = logits.view(B*T, C)
            target = target.view(-1)
            loss = F.cross_entropy(logits_reshaped, target)

        return logits, loss

    def generate(self, x, predict_len):
        for step in range(predict_len):
            logits, _ = self(x)
            logits = logits[:, -1, :] # Draw the logits of last time step. Now the shape is (B, C)
            probs = F.softmax(logits, dim=1)
            char_new = torch.multinomial(probs, 1)
            x = torch.cat([x, char_new], dim=1)
        return x

model = BigramLanguageModel(len(chars))
model = model.to(device)
x, y = get_batch(train_data)
logits, loss = model(x, y)
print(logits.shape)
print(loss)

# Use .tolist() method to transform the tensor
print(''.join(decode(x[0].tolist())))
x_predicted = model.generate(x, 8)
print(''.join(decode(x_predicted[0].tolist())))

torch.Size([8, 8, 65])
tensor(4.8997, grad_fn=<NllLossBackward0>)
girl.

J
girl.

JlpkRcoiy


In [13]:
# Train the Bigram model

optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

epochs = 1000
for _ in tqdm(range(epochs)):
    x, y = get_batch(train_data)
    _, loss = model(x, y)
    optimizer.zero_grad(set_to_none=True) # Save memory, and faster
    loss.backward()
    optimizer.step()
        
print(loss)

100%|██████████| 10000/10000 [00:06<00:00, 1535.56it/s]

tensor(2.6425, grad_fn=<NllLossBackward0>)


In [14]:
x, y = get_batch(train_data)

# Use .tolist() method to transform the tensor
print(''.join(decode(x[0].tolist())))
x_predicted = model.generate(x, 100)
print(''.join(decode(x_predicted[0].tolist())))

ff, I ha
ff, I hau
NEOTold!Ys'te d
CHQPA: D: fr;
AUKENCUp? wht tei'

Aupenerisonene:
ALLICr cac andr cal

wrhesay d n


# 计算attention的向量化方法

In [76]:
# For a (B, T, C) tensor
# By averaging over the prefix tokens to summarize the past info and ignore the future info (just like autoregressive)
# Ver1: Double loops version
# Track the dim can help coding

torch.manual_seed(42)

B = 4
T = 4
C = 3
x = torch.randn((B, T, C))

print(x[0])

x_loop = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        x_loop[b][t] = torch.mean(xprev, 0)

print(x_loop[0])

tensor([[ 1.9269,  1.4873,  0.9007],
        [-2.1055,  0.6784, -1.2345],
        [-0.0431, -1.6047, -0.7521],
        [ 1.6487, -0.3925, -1.4036]])
tensor([[ 1.9269,  1.4873,  0.9007],
        [-0.0893,  1.0829, -0.1669],
        [-0.0739,  0.1870, -0.3620],
        [ 0.3568,  0.0421, -0.6224]])


In [77]:
# Ver2: Matrix mul version. Efficient, using lower triangle torch.tril(torch.ones(3, 3)) and normalize it
# Use torch.allclose() to check these two

wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
print(wei)
x_MM = wei @ x
print(x_MM[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])
tensor([[ 1.9269,  1.4873,  0.9007],
        [-0.0893,  1.0829, -0.1669],
        [-0.0739,  0.1870, -0.3620],
        [ 0.3568,  0.0421, -0.6224]])


In [78]:
# Ver3: Set -inf (it is a mask) and then softmax to get the normalized lower triangular
# Why use this? A perspective of affinities. Weight matrix can represent how each token is interesting to each other (In this example it's just 0)
# Use [masked_fill] method
wei = torch.zeros((T, T))
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, value=-float('inf'))
wei = F.softmax(wei, dim=-1)
print(wei)
x_sm = wei @ x
print(x_sm[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500]])
tensor([[ 1.9269,  1.4873,  0.9007],
        [-0.0893,  1.0829, -0.1669],
        [-0.0739,  0.1870, -0.3620],
        [ 0.3568,  0.0421, -0.6224]])


In [82]:
# Ver4: Self-attention
# query * key to get the weights that represents the affinities(relationship) from token to token
# When training, the model will automatically learn the affinities from token to token.
# A simple one head attention.

"""
Important notes about atttention:
1. a communication machanism that can be describe by direct graph with block_size node (the edge represents the affinities.
    not necessarily to be a lower triangle. Encoder part needs tokens to talk to each other.
2. no notion of space. positional code is needed.
3. Raw value x is like private data, and V is like public data for aggregation
4. Self-attention: QKV from same source x. Cross-attention: outer source QK.
"""
torch.manual_seed(42)
head_size = 5
Q = nn.Linear(C, head_size)
K = nn.Linear(C, head_size)
V = nn.Linear(C, head_size)

q = Q(x) # (B, T, head_size)
k = K(x)

wei = q @ k.transpose(-2, -1) # (B, T, T)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, value=-float('inf')) * head_size**(-0.5)
wei = F.softmax(wei, dim=-1)
print(wei[0])

v = V(x)
print(v[0])
x_att = wei @ v # A good way of thinking this mat mul is row perspective
print(x_att[0])

tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.1491, 0.8509, 0.0000, 0.0000],
        [0.6266, 0.0590, 0.3144, 0.0000],
        [0.2801, 0.0278, 0.2398, 0.4523]], grad_fn=<SelectBackward0>)
tensor([[ 0.5241,  1.4449,  1.0499, -1.2772, -0.4092],
        [-0.4937, -0.3647, -1.0037,  1.0804,  0.6383],
        [ 0.3009, -0.1853, -0.4107,  1.5519, -1.2764],
        [ 0.2153,  1.2557,  0.0874,  0.4091, -1.5597]],
       grad_fn=<SelectBackward0>)
tensor([[ 0.5241,  1.4449,  1.0499, -1.2772, -0.4092],
        [-0.3419, -0.0948, -0.6974,  0.7288,  0.4821],
        [ 0.3938,  0.8256,  0.4695, -0.2486, -0.6200],
        [ 0.3026,  0.9182,  0.2073,  0.2294, -1.1084]],
       grad_fn=<SelectBackward0>)


In [80]:
# Why scaling using query and key dimension matters? 
# We wants the attention to combine more info. If not scaling, it will tend to focus on the largest one.
# Essentially, by scalling, the final weight matrix will have low variance.
example = torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])
print(torch.softmax(example, dim=-1))
print(torch.softmax(example * 8, dim=-1)) # Not what we want

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])
tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])


In [20]:
# Implement a self-attention Head
# 'tril' pytorch naming convention

In [21]:
# Implement a multi-head attention with projection (Linear trans)

In [22]:
"""
Modification on BigramModel -- 3 Linear layers
1. token_embedding layer (vocab_size, embed_size)
2. position_embedding layer (block_size, embed_size)
3. language model head used to compute logits (embed_size, block_size)
4. Pluggin the attention block (use multi-head directly)
5. Pluggin the last feedforward layer
"""

'\nModification on BigramModel -- 3 Linear layers\n1. token_embedding layer (vocab_size, embed_size)\n2. position_embedding layer (block_size, embed_size)\n3. language model head used to compute logits (embed_size, block_size)\n4. Pluggin the attention block (use multi-head directly)\n5. Pluggin the last feedforward layer\n'

In [23]:
"""
Implement attention block class with residual connection
"""

'\nImplement attention block class with residual connection\n'